# Model Preparation
A notebook that aids the preparation of a CNN.

In [11]:
# Imports
import os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix, average_precision_score, accuracy_score, precision_score, recall_score
models = tf.keras.models  # like 'from tensorflow.keras import models' (PyCharm import issue workaround)
layers = tf.keras.layers  # like 'from tensorflow.keras import layers' (PyCharm import issue workaround)
optimizers = tf.keras.optimizers  # like 'from tensorflow.keras import optimizers' (PyCharm import issue workaround)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Scripts import Data_Loader_Functions as DL
from Scripts import Centralized_Pain_CNN as painCNN

In [12]:
def build_cnn(input_shape):
    """
    Compile and return a simple CNN model for image recognition.

    Configuration:
    Layer 1: Convolution Layer | Filters: 32 | Kernel Size: 3x3 | Activation: Relu
    Layer 2: Max Pooling Layer | Filter: 2x2
    Layer 3: Dense Layer       | Neurons: 32 | Activation: Relu
    Layer 4: Dense Layer       | Neurons: 10 | Activation: Softmax

    Optimizer:      Adam
    Loss function:  Sparse Categorical Cross Entropy
    Loss metric:    Accuracy


    :param input_shape:     image input shape (tuple), e.g. (28, 28, 1)

    :return:
        model               compiled tensorflow model
    """

    # Set up model type
    model = models.Sequential()

    # Add layers
    model.add(layers.Conv2D(32, (5, 5), input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (5, 5)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (5, 5)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))

    return model

In [5]:
# Load data
path = os.path.join(module_path, 'Data', 'Augmented Data', 'Pain Two-Step Augmentation', 'group_1', 'p121')
path_2 = os.path.join(module_path, 'Data', 'Augmented Data', 'Pain Two-Step Augmentation', 'group_2_test', 'p048')
model = build_cnn((215, 215, 1))
test_data, test_labels = DL.load_pain_data(path)
test_data_2, test_labels_2 = DL.load_pain_data(path_2)
test_data = np.concatenate((test_data, test_data_2))
test_labels = np.concatenate((test_labels, test_labels_2))
label = 4
person = 0
test_labels_ord = test_labels[:, label].astype(np.int)
test_labels_bin = DL.reduce_pain_label_categories(test_labels_ord, max_pain=1)
people = test_labels[:, person].astype(np.int)
test_labels = test_labels_bin

0 images processed
0 images processed


In [7]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
predictions = model.predict(test_data)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
loss = loss(
            tf.convert_to_tensor(tf.cast(test_labels, tf.float32)),
            tf.convert_to_tensor(tf.cast(predictions, tf.float32))
        ).numpy()

y_pred = np.argmax(predictions, axis=1)

In [15]:
def compute_aggregate_metrics(epoch, loss, test_labels, y_pred, predictions):
    predictions = np.max(predictions, axis=1)

    # Getting relevant metrics
    accuracy = accuracy_score(test_labels, y_pred)
    recall = recall_score(test_labels, y_pred)
    precision = precision_score(test_labels, y_pred)
    tn, fp, fn, tp = confusion_matrix(test_labels, y_pred).ravel()
    group_avg_precision = average_precision_score(test_labels, predictions)
    results = [epoch, loss, accuracy, recall, precision, group_avg_precision, tp, tn, fp, fn]
    print(results)

    # Create DF for Progress
    df = pd.DataFrame([results], columns=['Epoch', 'Loss', 'Aggregate Accuracy', 'Aggregate Recall',
                                        'Aggregate Precision', 'Aggregate Avg. Precision', 'TP', 'TN', 'FP', 'FN'])
    df['Aggregate F1_Score'] = 2 * ((df['Aggregate Precision'] * df['Aggregate Recall']) / (df['Aggregate Precision'] +
                                                                                            df['Aggregate Recall']))
    return df


In [17]:
df = compute_aggregate_metrics(1, loss, test_labels, y_pred, predictions)

[1, 0.65701795, 0.6815816857440167, 0.0, 0.0, 0.2858647434577851, 0, 655, 14, 292]


In [18]:
df.columns

Index(['Epoch', 'Loss', 'Aggregate Accuracy', 'Aggregate Recall',
       'Aggregate Precision', 'Aggregate Avg. Precision', 'TP', 'TN', 'FP',
       'FN', 'Aggregate F1_Score'],
      dtype='object')